In [3]:
import numpy as np
import awkward as ak
import uproot
import vector
import mplhep as hep
import matplotlib.pyplot as plt
import os

In [1]:
def MakeJets(events):
    Jets = vector.zip({
            "pt"  : events["Jet_pt"].array(),
            "eta" : events["Jet_eta"].array(),
            "phi" : events["Jet_phi"].array(), 
            "mass": events["Jet_mass"].array(),
            "nTracks": events["Jet_nConstituents"].array(),
            "genWeight": events["genWeight"].array(),
        })
    return Jets

def GetMaxMjj(Jets):
    JetCombo = ak.combinations(Jets,2, fields=["jet1","jet2"])
    jjCombo  = JetCombo.jet1 + JetCombo.jet2
    mjjCombo = np.sqrt(np.abs((jjCombo.E)**2 - (jjCombo.px**2 + jjCombo.py**2 + jjCombo.pz**2)))
    maxmjj   = ak.max(mjjCombo,axis=1)
    return maxmjj

def GetMaxEta(Jets):
    JetCombo = ak.combinations(Jets,2, fields=["jet1","jet2"])
    etacombo = vector.Spatial.deltaeta(JetCombo.jet1, JetCombo.jet2)
    maxeta   = ak.max(etacombo, axis=1)
    return maxeta

def ApplyJetCuts(Jets):
    nJetCut = (ak.num(Jets)>=2)
    Jets = Jets[nJetCut]
    return Jets

def CheatVBFJets(events):
    Jets = MakeJets(events)
    GenParts = vector.zip({
        "pt": events["GenPart_pt"].array(),
        "eta": events["GenPart_eta"].array(),
        "phi": events["GenPart_phi"].array(),
        "mass": events["GenPart_mass"].array(),
    })

    fromQ, IsHiggs = events["GenPart_genPartIdxMother"].array(), events["GenPart_pdgId"].array()
    nJetCut = (ak.num(Jets)>=2)
    Jets, GenParts, fromQ, IsHiggs = Jets[nJetCut], GenParts[nJetCut], fromQ[nJetCut], IsHiggs[nJetCut]
    IsMotherProton = (fromQ==0) & (IsHiggs!=25)
    MotherProton = GenParts[IsMotherProton]
    JetMomCombo = ak.cartesian({"mom": MotherProton,"jet": Jets})
    dR = vector.Spatial.deltaR(JetMomCombo.mom, JetMomCombo.jet)
    min_dR = ak.where((dR-ak.sort(dR)[:,1])<0.0001, True, False)
    VBFJets = JetMomCombo.jet[min_dR]

    return VBFJets

In [4]:
# Caution!! The signal I am using in local machine is from Summer22!!!
sigPath = "/Users/raymondkil/Desktop/VBF Trigger/RootFiles/SUEP/"
ewkPath = "/Users/raymondkil/Desktop/VBF Trigger/RootFiles/EWK/"
qcdPath = "/Users/raymondkil/Desktop/VBF Trigger/RootFiles/QCD/"

sigFileNames, ewkFileNames, qcdFileNames = os.listdir(sigPath), os.listdir(ewkPath), os.listdir(qcdPath)
sigFileNames, ewkFileNames, qcdFileNames = [x for x in sigFileNames if x != '.DS_Store'], [x for x in ewkFileNames if x != '.DS_Store'], [x for x in qcdFileNames if x != '.DS_Store']

sigmaxmjjs, sigmaxetas, sigVBFetas, sigVBFmjjs = [],[],[],[]
ewkmaxmjjs, ewkmaxetas, ewkVBFetas, ewkVBFmjjs = [],[],[],[]
qcdmaxmjjs, qcdmaxetas, qcdVBFetas, qcdVBFmjjs = [],[],[],[]

processes = {
    "sig": [sigFileNames, sigPath, sigmaxmjjs, sigmaxetas, sigVBFetas, sigVBFmjjs],
    "ewk": [ewkFileNames, ewkPath, ewkmaxmjjs, ewkmaxetas, ewkVBFetas, ewkVBFmjjs],
    "qcd": [qcdFileNames, qcdPath, qcdmaxmjjs, qcdmaxetas, qcdVBFetas, qcdVBFmjjs]
}

FileNotFoundError: [Errno 2] No such file or directory: '/Users/raymondkil/Desktop/VBF Trigger/RootFiles/SUEP/'

In [5]:
for process in ['sig','ewk','qcd']:
    filenames, path, maxmjjs, maxetas, VBFetas, VBFmjjs = processes[process]
    for filename in filenames:
        print("Processing file {}".format(filename))
        f = uproot.open(path+filename)
        events = f["Events"]

        Jets = vector.zip({
            "pt"  : events["Jet_pt"].array(),
            "eta" : events["Jet_eta"].array(),
            "phi" : events["Jet_phi"].array(), 
            "mass": events["Jet_mass"].array(),
            "nTracks": events["Jet_nConstituents"].array(),
            "genWeight": events["genWeight"].array(),
        })
        Jets = ApplyJetCuts(Jets) # Cuts work well at this point

        maxmjj = GetMaxMjj(Jets)
        maxeta = GetMaxEta(Jets)

        VBFJet1, VBFJet2 = CheatVBFJets(events)[:,0], CheatVBFJets(events)[:,1]
        VBFeta = vector.Spatial.deltaeta(VBFJet1, VBFJet2)
        VBFjjCombo  = VBFJet1 + VBFJet2
        VBFmjj = np.sqrt(np.abs((VBFjjCombo.E)**2 - (VBFjjCombo.px**2 + VBFjjCombo.py**2 + VBFjjCombo.pz**2)))

        maxmjjs = np.concatenate((maxmjjs, maxmjj))
        maxetas = np.concatenate((maxetas, maxeta))
        VBFetas = np.concatenate((VBFetas, VBFeta))
        VBFmjjs = np.concatenate((VBFmjjs, VBFmjj))

    if process == 'sig':
        sigmaxmjjs, sigmaxetas, sigVBFetas, sigVBFetas = maxmjjs, maxetas, VBFetas, VBFmjjs
    if process == 'ewk':
        ewkmaxmjjs, ewkmaxetas, ewkVBFetas, ewkVBFetas = maxmjjs, maxetas, VBFetas, VBFmjjs
    if process == 'qcd':
        qcdmaxmjjs, qcdmaxetas, qcdVBFetas, qcdVBFetas = maxmjjs, maxetas, VBFetas, VBFmjjs

NameError: name 'processes' is not defined

In [6]:
# Weights
qcdWeights = []
for filename in qcdFileNames:
    print("Processing file {}".format(filename))
    f = uproot.open(path+filename)
    events = f["Events"]
    genWeight = events["genWeight"].array() # Looks like [1.25e-05, 1.32e-16, 1.55e-17, 1.25e-09, ... 1.23e-15, 2.21e-11, 5.11e-11, 5.48e-11], len 774000 -> one per event
    Jets = vector.zip({
        "pt"  : events["Jet_pt"].array(),
        "eta" : events["Jet_eta"].array(),
        "phi" : events["Jet_phi"].array(), 
        "mass": events["Jet_mass"].array(),
    })
    nJetCut = (ak.num(Jets)>=2) # looks like [True, True, True, True, True, True, True, ... True, True, True, True, True, True]
    genWeight = genWeight[nJetCut] # now len 758915 -> cut works well!
    qcdWeights = np.concatenate((qcdWeights, genWeight))

NameError: name 'qcdFileNames' is not defined

In [7]:
# Plotting for mjj
plt.figure()
binsize = 100
sigMAXhist, sigMAXbins = np.histogram(sigmaxmjjs, bins=np.arange(min(sigmaxmjjs), max(sigmaxmjjs) + binsize, binsize))
ewkMAXhist, ewkMAXbins = np.histogram(ewkmaxmjjs, bins=np.arange(min(ewkmaxmjjs), max(ewkmaxmjjs) + binsize, binsize))
qcdMAXhist, qcdMAXbins = np.histogram(qcdmaxmjjs, bins=np.arange(min(qcdmaxmjjs), max(qcdmaxmjjs) + binsize, binsize), weights=qcdWeights)
sigVBFhist, sigVBFbins = np.histogram(sigVBFmjjs, bins=np.arange(min(sigVBFmjjs), max(sigVBFmjjs) + binsize, binsize))
ewkVBFhist, ewkVBFbins = np.histogram(ewkVBFmjjs, bins=np.arange(min(ewkVBFmjjs), max(ewkVBFmjjs) + binsize, binsize))
qcdVBFhist, qcdVBFbins = np.histogram(qcdVBFmjjs, bins=np.arange(min(qcdVBFmjjs), max(qcdVBFmjjs) + binsize, binsize), weights=qcdWeights)
plt.style.use(hep.style.CMS)

hep.histplot(sigMAXhist, sigMAXbins, histtype='step', label='sig max', color='tab:blue',   density=True)
hep.histplot(ewkMAXhist, ewkMAXbins, histtype='step', label='ewk max', color='tab:green',  density=True)
hep.histplot(qcdMAXhist, qcdMAXbins, histtype='step', label='qcd max', color='tab:orange', density=True)
hep.histplot(sigVBFhist, sigVBFbins, histtype='step', label='sig vbf', color='tab:blue',   density=True, linestyle=('dashed'))
hep.histplot(ewkVBFhist, ewkVBFbins, histtype='step', label='ewk vbf', color='tab:green',  density=True, linestyle=('dashed'))
hep.histplot(qcdVBFhist, qcdVBFbins, histtype='step', label='qcd vbf', color='tab:orange', density=True, linestyle=('dashed'))

hep.cms.text("Simulation")
hep.cms.lumitext(r"Run3 Summer 22, (13 TeV)")
plt.title("")
plt.xlabel(r"$m_{jj}$ (GeV)")
plt.ylabel(r"Normalized Counts")
plt.xlim(1000,5000)
plt.ylim(0,0.001)
plt.legend()

NameError: name 'sigmaxmjjs' is not defined

<Figure size 640x480 with 0 Axes>

In [8]:
# Plotting etas

plt.figure()
binsize = 0.1
sigMAXhist, sigMAXbins = np.histogram(sigmaxetas, bins=np.arange(min(sigmaxetas), max(sigmaxetas) + binsize, binsize))
ewkMAXhist, ewkMAXbins = np.histogram(ewkmaxetas, bins=np.arange(min(ewkmaxetas), max(ewkmaxetas) + binsize, binsize))
qcdMAXhist, qcdMAXbins = np.histogram(qcdmaxetas, bins=np.arange(min(qcdmaxetas), max(qcdmaxetas) + binsize, binsize), weights=qcdWeights)
sigVBFhist, sigVBFbins = np.histogram(sigVBFetas, bins=np.arange(min(sigVBFetas), max(sigVBFetas) + binsize, binsize))
ewkVBFhist, ewkVBFbins = np.histogram(ewkVBFmjjs, bins=np.arange(min(ewkVBFetas), max(ewkVBFetas) + binsize, binsize))
qcdVBFhist, qcdVBFbins = np.histogram(qcdVBFmjjs, bins=np.arange(min(qcdVBFetas), max(qcdVBFetas) + binsize, binsize), weights=qcdWeights)
plt.style.use(hep.style.CMS)

hep.histplot(sigMAXhist, sigMAXbins, histtype='step', label='sig max', color='tab:blue',   density=True)
hep.histplot(ewkMAXhist, ewkMAXbins, histtype='step', label='ewk max', color='tab:green',  density=True)
hep.histplot(qcdMAXhist, qcdMAXbins, histtype='step', label='qcd max', color='tab:orange', density=True)
hep.histplot(sigVBFhist, sigVBFbins, histtype='step', label='sig vbf', color='tab:blue',   density=True, linestyle=('dashed'))
hep.histplot(ewkVBFhist, ewkVBFbins, histtype='step', label='ewk vbf', color='tab:green',  density=True, linestyle=('dashed'))
hep.histplot(qcdVBFhist, qcdVBFbins, histtype='step', label='qcd vbf', color='tab:orange', density=True, linestyle=('dashed'))

hep.cms.text("Simulation")
hep.cms.lumitext(r"Run3 Summer 22, (13 TeV)")
plt.title("")
plt.xlabel(r"$\Delta\eta$")
plt.ylabel(r"Normalized Counts")
plt.xlim(0,11)
plt.legend()

NameError: name 'sigmaxetas' is not defined

<Figure size 640x480 with 0 Axes>